# Начальные условия

Предполагается, что имеется файл artists&urls.txt, в котором отображено имя артиста и его страница на сайте с текстами песен genius.com

In [4]:
with open ('artists&urls.txt', encoding = 'UTF-8') as f:
    file = f.readlines()
print(file)
    

['Eminem https://genius.com/artists/Eminem\n', 'ABBA https://genius.com/artists/Abba\n', 'Adele https://genius.com/artists/Adele\n', 'Shakira https://genius.com/artists/Shakira\n', 'Backstreet Boys https://genius.com/artists/Backstreet-boys\n', 'The Beatles https://genius.com/artists/The-beatles\n', 'Billie Eilish https://genius.com/artists/Billie-eilish\n', 'Chase Atlantic https://genius.com/artists/Chase-atlantic\n', 'The Cranberries https://genius.com/artists/The-cranberries\n', 'Coldplay https://genius.com/artists/Coldplay\n', 'Elton John https://genius.com/artists/Elton-john\n', 'Evanescene https://genius.com/artists/Evanescence\n', 'Frank Sinatra https://genius.com/artists/Frank-sinatra\n', 'Gorillaz https://genius.com/artists/Gorillaz\n', 'Jaydes https://genius.com/artists/Jaydes\n', 'Led Zeppelin https://genius.com/artists/Led-zeppelin\n', 'Lil Peep https://genius.com/artists/Lil-peep\n', 'Metallica https://genius.com/artists/Metallica\n', 'Michael Jackson https://genius.com/ar

Далее извлечём из этого файла все url, установим нижний регистр всем буквам и озбавимся от ненужного \n в конце

In [5]:
urls = []

for i in range(len(file)):
    file[i] = file[i].lower()
    s = 0
    while (s != -1):
        s = file[i].find(' ')
        if (s != -1):
            file[i] = file[i][s+1:]
    if (file[i] != file[-1]):
        file[i] = file[i][:-1]
        urls.append(file[i])

print(urls)

['https://genius.com/artists/eminem', 'https://genius.com/artists/abba', 'https://genius.com/artists/adele', 'https://genius.com/artists/shakira', 'https://genius.com/artists/backstreet-boys', 'https://genius.com/artists/the-beatles', 'https://genius.com/artists/billie-eilish', 'https://genius.com/artists/chase-atlantic', 'https://genius.com/artists/the-cranberries', 'https://genius.com/artists/coldplay', 'https://genius.com/artists/elton-john', 'https://genius.com/artists/evanescence', 'https://genius.com/artists/frank-sinatra', 'https://genius.com/artists/gorillaz', 'https://genius.com/artists/jaydes', 'https://genius.com/artists/led-zeppelin', 'https://genius.com/artists/lil-peep', 'https://genius.com/artists/metallica', 'https://genius.com/artists/michael-jackson']


Далее переходим по странице и забираем его популярные песни для дальнейшего формирования Url запроса на текст этой песни. Соответственно нормализуем название песни: приводим в нижний регистр, вместо пробелов устанавливается тире, избавляемся от апострофа. Формируем словарь `имя артиста: [песни]`

In [18]:
import re
from bs4 import BeautifulSoup  
import requests  

# Словарь для хранения результатов
artist_songs_dict = {}

for url in urls:
    response = requests.get(url)

    # Проверка статуса ответа
    if response.status_code == 200:
        print("Страница успешно загружена")
        # Парсинг содержимого страницы
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Извлечение информации о первом артисте
        artist_link = soup.find('h1', class_='profile_identity-name_iq_and_role_icon u-hair_bottom_margin')
        if artist_link:
            text = artist_link.text
            a = text.find('\n')
            artist_name = artist_link.text[:a].strip()  # Убираем лишние пробелы
            print("Имя исполнителя:", artist_name)
        else:
            print("Исполнитель не найден.")
            continue
        
        # Приведение имени исполнителя к единому формату для фильтрации
        filter_list = {artist_name.lower()}

        print("Популярные песни этого исполнителя:")
        artist_songs = soup.find('div', class_='mini_card_grid')
        if artist_songs:
            # Получение текста и фильтрация пустых строк
            songs = [line.strip() for line in artist_songs.text.splitlines() if line.strip()]
            processed_songs = []
            for song in songs:
                if song.lower() not in filter_list:
                    formatted_song = re.sub(r'[^\w\s\']', '-', song)  # Заменяем всё, кроме букв, цифр, пробелов и апострофа, на дефис
                    formatted_song = formatted_song.replace("'", "")  # Убираем символ апострофа
                    formatted_song = re.sub(r'\s+', '-', formatted_song)  # Заменяем пробелы на дефисы
                    formatted_song = re.sub(r'-+', '-', formatted_song)  # Убираем последовательности из двух и более дефисов
                    formatted_song = formatted_song.strip('-')  # Убираем дефисы в начале и конце строки
                    formatted_song = formatted_song.lower()  # Приводим к нижнему регистру
                    if re.match(r'^[a-z0-9-]+$', formatted_song) and formatted_song not in filter_list:
                        print(formatted_song)
                        processed_songs.append(formatted_song)

            # Добавляем исполнителя и его песни в словарь
            
            if artist_name in artist_songs_dict:
                artist_songs_dict[artist_name].extend(processed_songs)
            else:
                artist_songs_dict[artist_name] = processed_songs

# Вывод результата
print("Словарь артистов и их песен:")
for artist, songs in artist_songs_dict.items():
    print(f"{artist}: {songs}")


Страница успешно загружена
Имя исполнителя: Eminem
Популярные песни этого исполнителя:
rap-god
lose-yourself
without-me
mockingbird
the-real-slim-shady
killshot
stan
godzilla
houdini
the-monster
Страница успешно загружена
Имя исполнителя: ABBA
Популярные песни этого исполнителя:
slipping-through-my-fingers
mamma-mia
dancing-queen
dancing-queen-live-1986
chiquitita
the-winner-takes-it-all
gimme-gimme-gimme-a-man-after-midnight
lay-all-your-love-on-me
happy-new-year
waterloo
Страница успешно загружена
Имя исполнителя: Adele
Популярные песни этого исполнителя:
hello
easy-on-me
rolling-in-the-deep
set-fire-to-the-rain
someone-like-you
when-we-were-young
all-i-ask
send-my-love-to-your-new-lover
skyfall
to-be-loved
Страница успешно загружена
Имя исполнителя: Shakira
Популярные песни этого исполнителя:
waka-waka-this-time-for-africa
hips-dont-lie
shakira-bzrp-music-sessions-vol-53
bizarrap-shakira
whenever-wherever
chantaje
she-wolf
tqg
karol-g-shakira
cant-remember-to-forget-you
la-la-la-bra

Также форматируем теперь имя артиста для каждого артиста в словаре (нижний регистр, избавление от апострофов, пробелы заменяются на тире) и сохраняем получившийся словарик в текстовый файл `artists_and_songs.txt`

In [19]:
def format_text(text):
    # Заменяем всё, кроме букв, цифр, пробелов и апострофа, на дефис
    formatted_text = re.sub(r'[^\w\s\']', '-', text)
    # Убираем апострофы
    formatted_text = formatted_text.replace("'", "")
    # Заменяем пробелы на дефисы
    formatted_text = re.sub(r'\s+', '-', formatted_text)
    # Убираем последовательности из двух и более дефисов
    formatted_text = re.sub(r'-+', '-', formatted_text)
    # Убираем дефисы в начале и конце строки
    formatted_text = formatted_text.strip('-')
    # Приводим к нижнему регистру
    return formatted_text.lower()

# Открытие файла в режиме записи (перезаписывает содержимое файла, если он уже существует)
with open('artists_and_songs.txt', 'w', encoding='utf-8') as file:
    for artist, songs in artist_songs_dict.items():
        # Запись артиста и его песен в файл
        file.write(f"{format_text(artist)}: {', '.join(songs)}\n")

print("Словарь успешно записан в файл artists_and_songs.txt")


Словарь успешно записан в файл artists_and_songs.txt


Дальше записываем себе все url на все песни из файла `artists_and_songs.txt`. В этом же файле внутренняя информация (имя артиста, название песни) начинаются с `#`, для дальнейшей удобной фильтрации этих строк. url формируется так, как ожидает его genius.com, то есть по правилу `https://genius.com/` + `имя_артиста` + `-` + `название песни` + `-lyrics`. 

In [1]:
# Открываем файл и читаем данные
with open('artists_and_songs.txt', 'r') as file:
    lines = file.readlines()

# Базовый URL
base_url = "https://genius.com/"

# Подготовка URL-адресов
formatted_data = []

for line in lines:
    # Убираем символы новой строки и разбиваем строку по разделителю ":"
    line = line.strip()
    if not line:  # Пропускаем пустые строки
        continue
    artist, songs = line.split(':')
    artist = artist.strip().replace(' ', '-').lower()  # Преобразуем имя автора

    # Обрабатываем список песен
    formatted_data.append(f"# {artist.replace('-', ' ').title()}")  # Добавляем имя артиста
    songs = songs.split(',')
    for song in songs:
        song = song.strip().replace(' ', '-').lower()  # Преобразуем название песни
        url = f"{base_url}{artist}-{song}-lyrics"
        formatted_data.append(f"## {song.replace('-', ' ').title()}")  # Добавляем название песни
        formatted_data.append(url)  # Добавляем URL

# Записываем результаты в текстовый файл
with open('urls_to_song_texts.txt', 'w') as output_file:
    output_file.write('\n'.join(formatted_data))

print("URL-адреса успешно записаны в файл urls_to_song_texts.txt")

URL-адреса успешно записаны в файл urls_to_song_texts.txt


Дальше переходим по всем url из файла и записываем текст каждой песни, сохраняя в конечном файле имя исполнителя и название песни также со спец. символом `#`

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pymorphy2
from nltk.corpus import stopwords

# Загрузка списка стоп-слов
try:
    stopwords_ru = set(stopwords.words("russian"))
except LookupError:
    import nltk
    nltk.download('stopwords')
    stopwords_ru = set(stopwords.words("russian"))

# Инициализация pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Функция для очистки и нормализации текста
def preprocess_text(text):
    # Перевод в нижний регистр
    text = text.lower()
    # Удаление пунктуации, но сохранение апострофов
    text = re.sub(r"[^\w\s']", " ", text)  # Убираем всё, кроме букв, цифр, пробелов и апострофов
    text = re.sub(r"\d+", " ", text)       # Убираем цифры
    # Токенизация текста
    words = text.split()
    # Фильтрация стоп-слов и нормализация
    cleaned_words = [
        morph.parse(word)[0].normal_form
        for word in words
        if word not in stopwords_ru
    ]
    # Объединение слов обратно в строку
    return " ".join(cleaned_words)

# Функция для извлечения текста песни с Genius
def fetch_lyrics(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        lyrics_divs = soup.find_all("div", class_="Lyrics-sc-1bcc94c6-1 bzTABU")
        lyrics = "\n".join(
            div.get_text(separator="\n").strip() for div in lyrics_divs
        )
        # Фильтрация строк, исключая те, которые начинаются с "[" и заканчиваются на "]"
        filtered_lyrics = "\n".join(
            line for line in lyrics.strip().split("\n")
            if not (line.startswith("[") and line.endswith("]"))
        )
        return filtered_lyrics.strip()
    except Exception as e:
        print(f"Ошибка при обработке {url}: {e}")
        return ""

# Основной процесс
input_file = "urls_to_song_texts.txt"
output_file = "normalized_lyrics.txt"

with open(input_file, "r", encoding="utf-8") as urls_file, \
     open(output_file, "w", encoding="utf-8") as output:
    
    current_artist = ""
    current_song = ""

    for line in urls_file:
        line = line.strip()
        if not line:
            continue  # Пропускаем пустые строки
        
        # Определяем имя артиста и название песни
        if line.startswith("# "):
            current_artist = line[2:]  # Убираем "# "
            output.write(f"# {current_artist}\n")
        elif line.startswith("## "):
            current_song = line[3:]  # Убираем "## "
            output.write(f"## {current_song}\n")
        else:
            # Предполагаем, что строка содержит URL
            url = line
            print(f"Обрабатываю песню: {current_artist} - {current_song}")
            raw_lyrics = fetch_lyrics(url)
            
            if raw_lyrics:
                normalized_lyrics = preprocess_text(raw_lyrics)
                if normalized_lyrics:
                    output.write(normalized_lyrics + "\n\n")
                else:
                    print(f"Нормализованный текст пуст для: {current_artist} - {current_song}")
            else:
                print(f"Не удалось получить текст песни: {current_artist} - {current_song}")

print("Обработка завершена. Все нормализованные тексты сохранены в файл.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ruslan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Обрабатываю песню: Eminem - Rap God
Обрабатываю песню: Eminem - Lose Yourself
Обрабатываю песню: Eminem - Without Me
Обрабатываю песню: Eminem - Mockingbird
Обрабатываю песню: Eminem - The Real Slim Shady
Обрабатываю песню: Eminem - Killshot
Обрабатываю песню: Eminem - Stan
Обрабатываю песню: Eminem - Godzilla
Обрабатываю песню: Eminem - Houdini
Обрабатываю песню: Eminem - The Monster
Обрабатываю песню: Abba - Slipping Through My Fingers
Обрабатываю песню: Abba - Mamma Mia
Обрабатываю песню: Abba - Dancing Queen
Обрабатываю песню: Abba - Dancing Queen Live 1986
Обрабатываю песню: Abba - Chiquitita
Обрабатываю песню: Abba - The Winner Takes It All
Обрабатываю песню: Abba - Gimme Gimme Gimme A Man After Midnight
Обрабатываю песню: Abba - Lay All Your Love On Me
Обрабатываю песню: Abba - Happy New Year
Обрабатываю песню: Abba - Waterloo
Обрабатываю песню: Adele - Hello
Обрабатываю песню: Adele - Easy On Me
Обрабатываю песню: Adele - Rolling In The Deep
Обрабатываю песню: Adele - Set Fire 

Дальше считаем TF-IDF по всем текстам всех исполнителей:

In [10]:
import math
import re
from collections import defaultdict

# Функция для токенизации текста
def tokenize(text):
    # Используем регулярное выражение для поиска слов, состоящих из букв и цифр
    return re.findall(r'\b\w+\b', text.lower())  # \b - граница слова, \w+ - одно или более букв/цифр

# Функция для подсчета частоты слов (TF)
def compute_tf(word_dict, total_words):
    tf_dict = {}
    for word, count in word_dict.items():
        tf_dict[word] = count / total_words
    return tf_dict

# Функция для подсчета IDF
def compute_idf(documents):
    N = len(documents)
    idf_dict = {}
    all_words = set(word for doc in documents for word in doc)
    for word in all_words:
        doc_count = sum(1 for doc in documents if word in doc)
        idf_dict[word] = math.log(N / (1 + doc_count))
    return idf_dict

# Функция для подсчета TF-IDF
def compute_tfidf(tf_dict, idf_dict):
    tfidf_dict = {}
    for word, tf in tf_dict.items():
        tfidf_dict[word] = tf * idf_dict.get(word, 0)
    return tfidf_dict

# Чтение файла
with open('normalized_lyrics.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Предобработка и токенизация, исключая строки, начинающиеся с #
documents = [tokenize(line.strip()) for line in lines if line.strip() and not line.startswith("#")]

# Подсчет TF для каждого документа
tf_list = []
for doc in documents:
    word_count = len(doc)
    word_dict = {}
    for word in doc:
        word_dict[word] = word_dict.get(word, 0) + 1
    tf_list.append(compute_tf(word_dict, word_count))

# Подсчет IDF для всех слов
idf_dict = compute_idf(documents)

# Подсчет TF-IDF для каждого документа
tfidf_documents = [compute_tfidf(tf, idf_dict) for tf in tf_list]

# Суммирование TF-IDF по всем документам для получения общего TF-IDF
total_tfidf = defaultdict(float)
for tfidf in tfidf_documents:
    for word, score in tfidf.items():
        total_tfidf[word] += score

# Сортировка по общему TF-IDF
sorted_tfidf = sorted(total_tfidf.items(), key=lambda x: -x[1])[:50]

# Вывод топ-50 слов с общим TF-IDF
print("Топ-50 слов по общему TF-IDF:")
for word, score in sorted_tfidf:
    print(f"{word}: {score:.4f}")


Топ-50 слов по общему TF-IDF:
la: 2.2985
na: 1.9482
ah: 1.8663
ooh: 1.5959
oh: 1.4683
yeah: 1.2887
da: 0.9993
she: 0.9753
m: 0.9524
your: 0.8950
ba: 0.8428
do: 0.8284
why: 0.8004
let: 0.7935
never: 0.7908
you: 0.7809
baby: 0.7781
love: 0.7720
tric: 0.7713
we: 0.7692
re: 0.7523
me: 0.7204
hoo: 0.7140
are: 0.7037
heart: 0.7021
be: 0.6953
don: 0.6837
was: 0.6828
wanna: 0.6808
ve: 0.6755
all: 0.6736
my: 0.6645
they: 0.6474
no: 0.6442
sun: 0.6420
i: 0.6391
when: 0.6387
on: 0.6375
gonna: 0.6329
what: 0.6252
that: 0.6240
like: 0.6239
doo: 0.6210
say: 0.6142
this: 0.6120
t: 0.6085
there: 0.6058
bennie: 0.6032
way: 0.6003
is: 0.5953


Дальше считаем TF-IDF для каждой песни отдельно:

In [1]:
import math
import re
from collections import defaultdict

# Функция для токенизации текста
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

# Функция для подсчета частоты слов (TF)
def compute_tf(word_dict, total_words):
    tf_dict = {}
    for word, count in word_dict.items():
        tf_dict[word] = count / total_words
    return tf_dict

# Функция для подсчета IDF
def compute_idf(documents):
    N = len(documents)
    idf_dict = {}
    all_words = set(word for doc in documents for word in doc)
    for word in all_words:
        doc_count = sum(1 for doc in documents if word in doc)
        idf_dict[word] = math.log(N / (1 + doc_count))
    return idf_dict

# Функция для подсчета TF-IDF
def compute_tfidf(tf_dict, idf_dict):
    tfidf_dict = {}
    for word, tf in tf_dict.items():
        tfidf_dict[word] = tf * idf_dict.get(word, 0)
    return tfidf_dict

# Путь к файлу
file_path = 'normalized_lyrics.txt'

# Словарь: {артист: {название песни: текст песни}}
lyrics_dict = defaultdict(dict)

# Чтение и обработка файла
with open(file_path, 'r', encoding='utf-8') as file:
    current_artist = None
    current_song = None
    current_lyrics = []

    for line in file:
        line = line.strip()  # Убираем лишние пробелы и символы новой строки
        if line.startswith('#') and not line.startswith('##'):  # Указание на исполнителя
            if current_artist and current_song and current_lyrics:
                # Сохраняем предыдущую песню
                lyrics_dict[current_artist][current_song] = '\n'.join(current_lyrics)
            current_artist = line[1:].strip()  # Получаем имя исполнителя
            current_song = None
            current_lyrics = []
        elif line.startswith('##'):  # Указание на название песни
            if current_song and current_lyrics:
                # Сохраняем предыдущую песню
                lyrics_dict[current_artist][current_song] = '\n'.join(current_lyrics)
            current_song = line[2:].strip()  # Получаем название песни
            current_lyrics = []
        elif line:  # Строка текста песни
            current_lyrics.append(line)

    # Сохраняем последнюю песню
    if current_artist and current_song and current_lyrics:
        lyrics_dict[current_artist][current_song] = '\n'.join(current_lyrics)

# Для подсчета TF-IDF по исполнителям
for artist, songs in lyrics_dict.items():
    print(f"Исполнитель: {artist}")
    # Токенизация текста всех песен исполнителя
    documents = [tokenize(lyrics) for song, lyrics in songs.items()]
    
    # Подсчет IDF для всех песен этого исполнителя
    idf_dict = compute_idf(documents)
    
    # Подсчет TF и TF-IDF для каждой песни исполнителя
    for song, lyrics in songs.items():
        print(f"  Песня: {song}")
        
        # Токенизация текста песни
        words = tokenize(lyrics)
        word_count = len(words)
        word_dict = {}
        for word in words:
            word_dict[word] = word_dict.get(word, 0) + 1
        
        # Подсчет TF для этой песни
        tf_dict = compute_tf(word_dict, word_count)
        
        # Подсчет TF-IDF для этой песни
        tfidf_dict = compute_tfidf(tf_dict, idf_dict)
        
        # Сортировка по TF-IDF
        sorted_tfidf = sorted(tfidf_dict.items(), key=lambda x: -x[1])[:10]  # Топ-10 слов
        
        # Вывод топ-10 слов по TF-IDF для песни
        for word, score in sorted_tfidf:
            print(f"    {word}: {score:.4f}")

Исполнитель: Eminem
  Песня: Rap God
    boy: 0.0088
    lookin: 0.0081
    rap: 0.0078
    nod: 0.0059
    god: 0.0045
    slap: 0.0039
    box: 0.0039
    while: 0.0037
    minutes: 0.0029
    something: 0.0029
  Песня: Lose Yourself
    opportunity: 0.0151
    lifetime: 0.0113
    shot: 0.0097
    yo: 0.0086
    miss: 0.0085
    dah: 0.0076
    goes: 0.0075
    own: 0.0075
    knows: 0.0071
    better: 0.0067
  Песня: Without Me
    la: 0.0607
    na: 0.0474
    controversy: 0.0133
    round: 0.0114
    follow: 0.0114
    empty: 0.0114
    without: 0.0114
    job: 0.0099
    outside: 0.0085
    looks: 0.0085
  Песня: Mockingbird
    daddy: 0.0302
    mama: 0.0126
    always: 0.0063
    stiffen: 0.0050
    upper: 0.0050
    lip: 0.0050
    cry: 0.0050
    lainie: 0.0050
    mommy: 0.0050
    ring: 0.0050
  Песня: The Real Slim Shady
    stand: 0.0428
    please: 0.0320
    slim: 0.0191
    real: 0.0121
    yes: 0.0118
    shadys: 0.0118
    imitating: 0.0118
    shady: 0.0098
    won